In [ ]:
%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# 返回指定路径下的文件和文件夹列表
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 导入后面要使用的包/库

In [ ]:
# !python      #查看python 版本

In [ ]:
# !python      #查看python 版本
# !nvcc --version   #查看cuda版本


In [ ]:
 import tensorflow as tf
tf.__version__

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt
# io.imread()是图片读取    io.imshow()是图片显示
import skimage.io
# from skimage import io,data
# img = io.imread('../input/resized-2015-2019-blindness-detection-images/resized train 15/10029_left.jpg')
# io.imshow(img)
# print ('img的宽度：', img.shape[0])  #图片宽度
# print ('img的高度：', img.shape[1])  #图片高度
from skimage.transform import resize
# imgaug是一个数据增强的库
from imgaug import augmenters as iaa
# tqdm是一个python进度条
from tqdm import tqdm
# import PIL用于导入可以产生各种处理图片的函数
import PIL
from PIL import Image, ImageOps
# opencv库是计算机视觉库，调用的时候经常import cv2
import cv2
# sklearn.utils.shuffle()用来打乱样本
# sklearn.utils.class_weight对训练集里的每个类别加一个权重。类别的样本数多，它的权重就低，反之则权重就高
from sklearn.utils import class_weight, shuffle
# tf.keras.losses实例是用来计算真实标签（ y_true ）和预测标签之间（ y_pred ）的loss损失
from keras.losses import binary_crossentropy, categorical_crossentropy
# 导入网络模型：已经训练好权重的resnet50      preprocess_input是为resnet网络准备新的图像输入
#                                         用法：image = preprocess_input(image)
from keras.applications.resnet50 import preprocess_input
# 导入backend模块
import keras.backend as K
import tensorflow as tf
# 导入各种评估指标函数
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score, accuracy_score
# 导入Sequence类，用keras.utils.Sequence()构建一个数据生成器
from keras.utils import Sequence 
# keras.utils.to_categorical将类别向量转换成只用二进制（0和1）表示的形式
from keras.utils import to_categorical
# 调用import train_test_split()函数来划分训练集和测试集
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")  #“ignore是忽略警告”有时候代码能运行出来但是会有警告
SIZE = 300
NUM_CLASSES = 5

In [ ]:
# pd.read_csv()接口用于读取csv格式数据文件
df_train = pd.read_csv('../input/resized-2015-2019-diabetic-retinopathy-detection/labels/trainLabels15.csv')
df_test = pd.read_csv('../input/resized-2015-2019-diabetic-retinopathy-detection/labels/testLabels15.csv')

x = df_train['image']
y = df_train['level']

# x， y= shuffle(x, y, random_state=1):把x和y一一对应后打乱，如果想要每次打乱后的排序都相同，就给
# random_state设置实值
x, y = shuffle(x, y, random_state=8)
y.hist()   #构建直方图



# 划分数据集

In [ ]:
# # to_categorical()中y是待转换的标签数组，num_classes是标签类别数，注意类别个数加一
# # https://blog.csdn.net/weixin_42886817/article/details/100118621
y = to_categorical(y, num_classes=NUM_CLASSES)  #num_classes可以设置为具体数值，但最好这样设置

# # train_x，train_y是原始数据集划分出来作为训练模型的
# # valid_x, valid_y用于评价训练出来的模型好坏，score评分的时候用
# # x:所要划分的样本特征集      y:所要划分的样本结果
# # test_size:“测试集”比例
# # stratify=y是为了保持和y split前 类的分布,通常在这种类分布不平衡的情况下会用到stratify
# # 即划分后的训练集和测试集里边的类别比例都保持和划分前的比例一样
# # random_state是0或者None的时候，每次划分的结果不一样，是具体数值的时候每次划分都一样
# train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,
#                                                       stratify=y, random_state=8)
# train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2,
#                                                       stratify=y, random_state=8)
# print(train_x.shape)
# print(train_y.shape)
# print(valid_x.shape)
# print(valid_y.shape)
# print(test_x.shape)
# print(test_y.shape)
# train_x，train_y是原始数据集划分出来作为训练模型的
# valid_x, valid_y用于评价训练出来的模型好坏，score评分的时候用
# x:所要划分的样本特征集      y:所要划分的样本结果
# test_size:“测试集”比例
# stratify=y是为了保持和y split前 类的分布,通常在这种类分布不平衡的情况下会用到stratify
# 即划分后的训练集和测试集里边的类别比例都保持和划分前的比例一样
# random_state是0或者None的时候，每次划分的结果不一样，是具体数值的时候每次划分都一样
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=8)
print(train_x.shape)
print(train_y.shape)
print(valid_x.shape)
print(valid_y.shape)

# 数据增强

In [ ]:
# https://github.com/aleju/imgaug
# iaa.Sometimes(0.5)有50%的图片会被应用一部分Augmenters，剩下的50%会被应用另外的Augmenters
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# 产生一个做数据增强的Sequential实例
seq = iaa.Sequential([
    sometimes(
#         iaa.OneOf()每次从一系列Augmenters中选择一个来变换
        iaa.OneOf([
#             iaa.Add(，per_channel=True)每个通道使用相同的 value,这里是在（-10~10）之间随机抽取一个数，
#             并保持之后都是这个数
#             iaa.Add(,per_channel=False)每个通道使用不同的 value，每抽取一次，这个值都会发生变化
#             iaa.Add(,per_channel=0.5)在50%的图像中，所有通道的值相同（这个值是所加的随机值）
#             在其他50%的图像中，每个通道的值不同，但是都会在-10~10之间
#             per_channel：每个通道是否使用相同的 value
         
            iaa.Add((-10, 10), per_channel=0.5),
#            iaa.Multiply() 给图像中的每个像素点乘一个值使得图片更亮或者更暗，这里同样是操作50%的图像
            iaa.Multiply((0.9, 1.1), per_channel=0.5),
#             将对比度标准化为0.9至1.1倍，对50%的图像，另外50%不变
#             可以把这里的对比度理解为一张图片色彩差异，差异越大，图像越清晰醒目，色彩也越鲜明艳丽
            iaa.ContrastNormalization((0.9, 1.1), per_channel=0.5)
        ])
    ),
    iaa.Fliplr(0.5),  #对50%的图像进行镜像翻转
    iaa.Crop(percent=(0, 0.1)),  #对图像的四个方向即四条边都裁剪0-10%，可以填充常数也可以填充随机值
    iaa.Flipud(0.5)   #对50%的图像进行左右翻转
],random_order=True)


# 优化

In [ ]:
# 在自定义优化器时，一般都会用到这三行代码
from keras.legacy import interfaces
from keras.optimizers import Optimizer
from keras import backend as K    #使用抽象keras backend API来写代码


class AdamAccumulate_v1(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, accum_iters=20, **kwargs):
        super(AdamAccumulate, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.effective_iterations = K.variable(0, dtype='int64', name='effective_iterations')

            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.accum_iters = K.variable(accum_iters, dtype='int64')

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)

        self.updates = [K.update(self.iterations, self.iterations + 1)]

        flag = K.equal(self.iterations % self.accum_iters, self.accum_iters - 1)
        flag = K.cast(flag, K.floatx())

        self.updates.append(K.update(self.effective_iterations,
                                     self.effective_iterations + K.cast(flag, 'int64')))

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.effective_iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.effective_iterations, K.floatx()) + 1

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        gs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat, gg in zip(params, grads, ms, vs, vhats, gs):

            gg_t = (1 - flag) * (gg + g)
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * (gg + flag * g) / K.cast(self.accum_iters, K.floatx())
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(
                (gg + flag * g) / K.cast(self.accum_iters, K.floatx()))

            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - flag * lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - flag * lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append((m, flag * m_t + (1 - flag) * m))
            self.updates.append((v, flag * v_t + (1 - flag) * v))
            self.updates.append((gg, gg_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(AdamAccumulate, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class AdamAccumulate(Optimizer):   #优化器设置
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, accum_iters=2, **kwargs):
        if accum_iters < 1:
            raise ValueError('accum_iters must be >= 1')
        super(AdamAccumulate, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.accum_iters = K.variable(accum_iters, K.dtype(self.iterations))
        self.accum_iters_float = K.cast(self.accum_iters, K.floatx())

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        completed_updates = K.cast(K.tf.floor(self.iterations / self.accum_iters), K.floatx())

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * completed_updates))

        t = completed_updates + 1

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))

        # self.iterations incremented after processing a batch
        # batch:              1 2 3 4 5 6 7 8 9
        # self.iterations:    0 1 2 3 4 5 6 7 8
        # update_switch = 1:        x       x    (if accum_iters=4)
        update_switch = K.equal((self.iterations + 1) % self.accum_iters, 0)
        update_switch = K.cast(update_switch, K.floatx())

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        gs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]

        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat, tg in zip(params, grads, ms, vs, vhats, gs):

            sum_grad = tg + g
            avg_grad = sum_grad / self.accum_iters_float

            m_t = (self.beta_1 * m) + (1. - self.beta_1) * avg_grad
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(avg_grad)

            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, (1 - update_switch) * vhat + update_switch * vhat_t))
            else:
                p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, (1 - update_switch) * m + update_switch * m_t))
            self.updates.append(K.update(v, (1 - update_switch) * v + update_switch * v_t))
            self.updates.append(K.update(tg, (1 - update_switch) * sum_grad))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, (1 - update_switch) * p + update_switch * new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(AdamAccumulate, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# 把数据喂进网络

In [ ]:
# Sequence类必须重载三个私有方法，__init__、__len__和__getitem__
# __init__初始化数据，让样本数据集通过形参顺利传进来
# __len__用于计算样本数据长度，也就是跑完一遍数据要运行模型多少次
# __getitem__生成批量数据，喂给神经网络模型训练
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels,
                 batch_size, is_train=False,  #is_train=False记录每一个Batch的均值和方差, 训练完成之后按照下式计算整体的均值和方差
                 mix=False, augment=False):    #是否混合，是否使用随机增强的样本
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.is_train = is_train
        self.is_augment = augment
        if(self.is_train):
            self.on_epoch_end()  #如果记录，那么就在每个epoch之后进行
        self.is_mix = mix

    def __len__(self):   #相当于计算batch：总长度/batch_size
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        if(self.is_train):
            return self.train_generate(batch_x, batch_y)
        return self.valid_generate(batch_x, batch_y)
    
# 跑完一个epoch后，想要实现什么，可以通过这个函数实现
# 比如这里，跑完一个epoch后对image和label进行一次清洗
    def on_epoch_end(self):
        if(self.is_train):
            self.image_filenames, self.labels = shuffle(self.image_filenames, self.labels)
    
    def mix_up(self, x, y):   #混合增强，x是样本，y是标签
        lam = np.random.beta(0.2, 0.4)  #认为lam是混合系数就可以，而混合系数就是通过0.2和0.4计算出的，有公式
        ori_index = np.arange(int(len(x)))  #认为ori_index和index_array分别是两个样本及其标签的表示
        index_array = np.arange(int(len(x)))
        np.random.shuffle(index_array)        
        
#         mixed_x = 系数 * ori_index +（1-系数） * index_array
#         mixed_x是混合后的样本，mixed_y是混合后的样本标签
        mixed_x = lam * x[ori_index] + (1 - lam) * x[index_array]
        mixed_y = lam * y[ori_index] + (1 - lam) * y[index_array]
        
        return mixed_x, mixed_y

    def train_generate(self, batch_x, batch_y):  #batch_x和batch_y是一个一个的batch
        batch_images = []  #创建一个空list列表
        for (sample, label) in zip(batch_x, batch_y):
            img = cv2.imread('../input/resized-2015-2019-diabetic-retinopathy-detection/resized_traintest15_train19/'+sample+'.jpg')
#             print(img.shape)
            img = cv2.resize(img, (SIZE, SIZE),interpolation=cv2.INTER_AREA)  #对图像大小进行缩放（size， size）分别是水平和竖直方向
            if(self.is_augment):
                img = seq.augment_image(img)
            batch_images.append(img)  #append() 方法将img添加到现有列表中
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        if(self.is_mix):
            batch_images, batch_y = self.mix_up(batch_images, batch_y)
        return batch_images, batch_y
   

    def valid_generate(self, batch_x, batch_y):
        batch_images = []
        for (sample, label) in zip(batch_x, batch_y):
            img = cv2.imread('../input/resized-2015-2019-diabetic-retinopathy-detection/resized_traintest15_train19/'+sample+'.jpg')
                
            img = cv2.resize(img, (SIZE, SIZE),interpolation=cv2.INTER_AREA)
                
                
            batch_images.append(img)
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y


In [ ]:
# ImageDataGenerator类：实时数据增强生成批量图像数据向量。训练时会无限循环生成数据，直到达到规定的epoch次数为止
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model    #load_model是加载模型
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50   #读取resnet50中的ResNet50
# 回调函数的使用https://keras.io/zh/callbacks/
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

# 网络模型

In [ ]:
function = "softmax"
# 创建一个模型
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(include_top=False,  #include_top：是否保留顶层的3个全连接网络
                   weights=None,  #参数随机初始化，不加载预训练模型
                   input_tensor=input_tensor)
    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation=function, name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

# 写一个LossHistory类，保存loss和acc

In [ ]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = {'batch':[], 'epoch':[]}
#         self.accuracy = {'batch':[], 'epoch':[]}
#         self.val_loss = {'batch':[], 'epoch':[]}
#         self.val_acc = {'batch':[], 'epoch':[]}

#     def on_batch_end(self, batch, logs={}):
#         self.losses['batch'].append(logs.get('loss'))
#         self.accuracy['batch'].append(logs.get('acc'))
#         self.val_loss['batch'].append(logs.get('val_loss'))
#         self.val_acc['batch'].append(logs.get('val_acc'))

#     def on_epoch_end(self, batch, logs={}):
#         self.losses['epoch'].append(logs.get('loss'))
#         self.accuracy['epoch'].append(logs.get('acc'))
#         self.val_loss['epoch'].append(logs.get('val_loss'))
#         self.val_acc['epoch'].append(logs.get('val_acc'))

#     def loss_plot(self, loss_type):
#         iters = range(len(self.losses[loss_type]))
#         plt.figure()
#         # acc
#         plt.plot(iters, self.accuracy[loss_type], 'r', label='test acc')
#         # loss
#         plt.plot(iters, self.losses[loss_type], 'g', label='test loss')
# #         if loss_type == 'epoch':
# #             # val_acc
# #             plt.plot(iters, self.val_acc[loss_type], 'b', label='test acc')
# #             # val_loss
# #             plt.plot(iters, self.val_loss[loss_type], 'k', label='test loss')
#         plt.grid(True)
#         plt.xlabel(loss_type)
#         plt.ylabel('acc-loss')
#         plt.legend(loc="upper right")
#         plt.show()
        


# 创建一个实例history

In [ ]:
# history = LossHistory()

In [ ]:
# create callbacks list
# 回调函数文档  https://keras.io/zh/callbacks/
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)

epochs = 20; batch_size = 32

# ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, 
#                 save_weights_only)
# filepath：保存模型的路径      monitor: 被监测的数据    verbose=0:不显示信息  verbose=1:显示进度条
#  save_best_only=True， 被监测数据的最佳模型就不会被覆盖    因为用的是val_loss,所以mode='min'
# save_weights_only = True只有模型的权重被保存
checkpoint = ModelCheckpoint('../working/Resnet50.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                   verbose=1, mode='min', epsilon=0.0001)
# 所以训练12个epoch就自动停止了
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=9)

csv_logger = CSVLogger(filename='../working/training_log.csv',
                       separator=',',  #用“,”进行隔离.csv文件中的元素字符串
                       append=True)  #不覆盖原有文件
# callbacks_list = [checkpoint, csv_logger, reduceLROnPlat, early]

train_generator = My_Generator(train_x, train_y, 128, is_train=True)
train_mixup = My_Generator(train_x, train_y, batch_size, is_train=True, mix=False, augment=True)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=NUM_CLASSES)  #n_out是输出数量，设置为NUM_CLASSES,比较方便

In [ ]:
# 有关QWK分数的计算
from keras.callbacks import Callback
class QWKEvaluation(Callback):
    def __init__(self, validation_data=(), batch_size=32, interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator,
                                                  steps=np.ceil(float(len(self.y_val)) / float(self.batch_size)),
                                                  workers=1, use_multiprocessing=True,
                                                  verbose=1)
            def flatten(y):
                return np.argmax(y, axis=1).reshape(-1)
                # return np.sum(y.astype(int), axis=1) - 1
            
            score = cohen_kappa_score(flatten(self.y_val),
                                      flatten(y_pred),
                                      labels=[0,1,2,3,4],
                                      weights='quadratic')
#             print(flatten(self.y_val)[:5])
#             print(flatten(y_pred)[:5])
            print("\n epoch: %d - QWK_score: %.6f \n" % (epoch+1, score))
            self.history.append(score)
            if score >= max(self.history):
                print('save checkpoint: ', score)
                self.model.save('../working/Resnet50_bestqwk.h5')

qwk = QWKEvaluation(validation_data=(valid_generator, valid_y),
                    batch_size=batch_size, interval=1)

In [ ]:
# # warm up model
# for layer in model.layers:
#     layer.rainable = False      #这段代码的意思是将所有层冻结，权重不会更新，

# for i in range(-5,0):
#     model.layers[i].trainable = True     #这是让从倒数第5到倒数第1层解冻，可以更新权重

# #     调用优化器：之前已经定义好了
# model.compile(        
#     loss='categorical_crossentropy',  #分类交叉熵，如【0，0，1】或【0.1, 0.2, 0.7】都是指的预测第三类
#     # loss='binary_crossentropy',  二元交叉熵，用于二分类{0，1}
#     optimizer=Adam(1e-3))

# model.fit_generator(
#     train_generator,
#     steps_per_epoch=np.ceil(float(len(train_y)) / float(128)),
#     epochs=3,
#     workers=WORKERS, use_multiprocessing=True,
#     verbose=1,
#     callbacks=[qwk])

# 训练网络

In [ ]:
# train all layers
for layer in model.layers:
    layer.trainable = True

callbacks_list = [checkpoint, csv_logger, reduceLROnPlat, early, qwk]
# model.compile()让模型解冻生效
model.compile(loss='categorical_crossentropy',
            # loss=kappa_loss,
            # loss='binary_crossentropy',
            # optimizer=Adam(lr=1e-4),
            optimizer=AdamAccumulate(lr=1e-4, accum_iters=2),
            metrics=['accuracy'])

# model.fit_generator(
#     train_mixup,
#     steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
#     validation_data=valid_generator,
#     validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
#     epochs=epochs,
#     verbose=1,
#     workers=1, use_multiprocessing=False,
#     callbacks=[history])
model.fit_generator(
    train_mixup,
    steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
    validation_data=valid_generator,
    validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
    epochs=epochs,
    verbose=1,
    workers=1, use_multiprocessing=False,
    callbacks=callbacks_list)

# 绘制acc-loss曲线

In [ ]:
# history.loss_plot('epoch')

# 测试

In [ ]:
submit = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
# model.load_weights('../working/Resnet50.h5')
# 从被保存的h5文件中,加载所有层的权重，其中h5文件里边有已经训练好的权重
model.load_weights('../working/Resnet50_bestqwk.h5')
predicted = []    #一个预测的空列表

In [ ]:
# tqdm是Python进度条，可以在Python 长循环中添加一个进度提示信息
for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('../input/aptos2019-blindness-detection/train_images/', name+'.png')
    image = cv2.imread(path)
    image = cv2.resize(image, (SIZE, SIZE))
    score_predict = model.predict((image[np.newaxis])/255)
    
    label_predict = np.argmax(score_predict)
    # label_predict = score_predict.astype(int).sum() - 1
#     在表格最后一栏添加“预测”
    predicted.append(str(label_predict))

In [ ]:
submit['diagnosis'] = predicted
# index=False：输出不显示索引
# submit.to_csv('submission.csv', index=True)
submit.to_csv('submission.csv', index=False)
submit.head()